# Cleaning Up the Ultra-Trail du Mont-Blanc (UTMB) Dataset
The inspiration for using this dataset came from the following Kaggle datasets:
- [Dataset: Ultra-Trail du Mont-Blanc 2003-2017](https://www.kaggle.com/ceruleansea/ultratrail-du-montblanc-20032017)

The original dataset taken from here: <https://utmbmontblanc.com/en/page/349/results.html>

## Importing Packages

In [ ]:
import numpy as np
import pandas as pd

## Deffining Parameters

In [ ]:
years = [2004, 2005, 2006, 2007, 2008, 2009, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
age_group_mapping = {
    'ES': 0,
    'SE': 1,
    'V1': 2,
    'V2': 3,
    'V3': 4,
    'V4': 5,
    'V5': 6,
}

## Downloading Dataset

Manually download the web pages containing the dataset as a table. Go to <https://utmbmontblanc.com/en/page/349/results.html> select each of the years, right click the web page and select "save as...". Save the pages in "../../static/datasets/original/utmb_pages/{year}.html"

In [ ]:
data_pages = '../../static/datasets/original/utmb_pages/{}.html'
dataset_file = '../../static/datasets/utmb.csv'

## Loading the Datset

In [ ]:


datasets_list = []
for year in years:
    dataset_tmp = pd.read_html(data_pages.format(year), attrs = {'id': 'tabPass'})[0]
    dataset_tmp = pd.DataFrame({'Rank': dataset_tmp[0],
                                'Name': dataset_tmp[2],
                                'Category': dataset_tmp[3],
                                'Year': year,
                                'Time': dataset_tmp[len(dataset_tmp.columns) - 1],
                                })
    dataset_tmp = dataset_tmp.dropna(axis=0)
    datasets_list.append(dataset_tmp)

full_dataset = pd.concat(datasets_list, axis=0, ignore_index=True)

## Displaying the first 10 rows of the dataset

In [ ]:
print(len(full_dataset))
full_dataset.head(10)

19516


,Rank,Name,Category,Year,Time
0,1,DELEBARRE Vincent,SE H,2004,21:06:18
1,2,SHERPA Dachhiri-Dawa,SE H,2004,23:02:28
2,3,PACHE Jean Claude,SE H,2004,23:40:08
3,4,FAVRE-FELIX Lionel,SE H,2004,23:50:36
4,5,NEMETH Csaba,SE H,2004,23:54:09
5,6,CERCUEIL Michel,SE H,2004,24:46:26
6,7,VERDIER Philippe,V1 H,2004,24:46:28
7,8,FATTON Christian,V1 H,2004,25:20:30
8,9,SCHWEIZER Werner,V3 H,2004,25:24:22
9,10,TECHER Henri,V1 H,2004,25:36:29


## Cleaning up the data

In [ ]:
dataset = full_dataset.copy()  # Creat a copy of the data

## convert time from string to float-hours
dataset['Result'] = pd.to_timedelta(dataset['Time']).astype(int) / 1e9 / 60 / 60
dataset['Age Group'] = dataset['Category'].apply(lambda x: age_group_mapping[x[:2]])
dataset['Gender'] = dataset['Category'].apply(lambda x: int(x[-1:] == 'F'))

## Remove unneccesery fields
dataset.pop('Category')
dataset.pop('Time')

## Print first 10 rows
dataset.head(10)

,Rank,Name,Year,Result,Age Group,Gender
0,1,DELEBARRE Vincent,2004,21.105000,1,0
1,2,SHERPA Dachhiri-Dawa,2004,23.041111,1,0
2,3,PACHE Jean Claude,2004,23.668889,1,0
3,4,FAVRE-FELIX Lionel,2004,23.843333,1,0
4,5,NEMETH Csaba,2004,23.902500,1,0
5,6,CERCUEIL Michel,2004,24.773889,1,0
6,7,VERDIER Philippe,2004,24.774444,2,0
7,8,FATTON Christian,2004,25.341667,2,0
8,9,SCHWEIZER Werner,2004,25.406111,4,0
9,10,TECHER Henri,2004,25.608056,2,0


## Save the clean dataset

In [ ]:
dataset.to_csv(dataset_file, index=False)